In [1]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = 'cpu'
print("Device:", device)

Device: cuda:0


In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# pipe_translate = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr", device=device) # Bad perfs
pipe_translate = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-fr", batch_size=16, device=device) # Decent perfs, quite fast
# pipe_translate = pipeline("translation", model="jbochi/madlad400-3b-mt", batch_size=16, device=device) # Verfy slow, haven't found the right conf to make it work properly
test_text = "Hello, I'm Quentin, how are you today? I have a medical problem I'd like to discuss."
translation = pipe_translate(test_text)
print("Translation:", translation)

/home/qbateux/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Translation: [{'translation_text': "2fr> Bonjour, je suis Quentin, comment allez-vous aujourd'hui? J'ai un problème médical dont je voudrais discuter."}]


In [55]:
import zipfile
from bs4 import BeautifulSoup
from tqdm import tqdm

file_name = "epubs/King of Scars by Leigh Bardugo.epub"
input_archive = zipfile.ZipFile(file_name, "r")
output_archive = zipfile.ZipFile(file_name+"_traduction.epub", "w")
file_list = input_archive.infolist()
pprint(file_list)

for x in tqdm(range(0, len(file_list))):
    item = input_archive.open(file_list[x])
    content = item.read()
    
    if file_list[x].filename.endswith(".html"):# and "0009" in file_list[x].filename:
        # modification = '''<?xml version='1.0' encoding='utf-8'?>
        #                 <html xmlns="http://www.w3.org/1999/xhtml">
        #                 <head>
        #                     <title>1 Dima</title>
                            
        #                 <link href="../Styles/stylesheet.css" rel="stylesheet" type="text/css"/>
        #                 <link href="../Styles/page_styles.css" rel="stylesheet" type="text/css"/>
        #                 </head>
        #                 <body id="ukwsaIexvXoTa2hwYWoxGfG" class="calibre">
        #                 <p class="linespace"> </p>
        #                 <div class="imgcenter">
        #                 <p class="chheadimg3"><img src="../Images/00007.jpeg" width="70%" height="auto" alt="images" class="calibre1"/></p>
        #                 </div>
        #                 <p class="linespace1"></p>
        #                 <p class="tx"><span class="big">DIMA HEARD THE BARN DOORS</span> slam before anyone else did. Inside the little farmhouse, the kitchen bubbled like a pot on the stove, its windows shut tight against the storm, the air in the room warm and moist. The walls rattled with the rowdy din of Dima’s brothers talking over one another, as his mother hummed and thumped her foot to a song Dima didn’t know. She held the torn sleeve of one of his father’s shirts taut in her lap, her needle pecking at the fabric in the uneven rhythm of an eager sparrow, a skein of wool thread trailing between her fingers like a choice worm.</p>
        #                 </body>
        #                 </html>
        #                 '''
        
        
        html = input_archive.read(file_list[x].filename)
        soup = BeautifulSoup(html, 'html.parser')
        for i in soup.find_all('p', ["tx", "txt"]):
            sentences = i.text.split(". ")
            
            # Making sure there are no words only in uppercase (otherwise then don't get translated...)
            for idx, sentence in enumerate(sentences):
                if "." not in sentence:
                    sentence = sentence + ". " # Adding the ending dot back. Otherwise some sentences don't make sense and get translated wrong
                    sentences[idx] = sentence
                found_upper = False
                for word in sentence.split(" "):
                    if word.isupper():
                        found_upper = True
                        sentence = sentence.replace(word, word.lower())
                        # print("Replaced Upper case words:", word, "by", word.lower())
                        # print("Updated sentence (ONGOING):", sentence)
                if found_upper:
                    sentence = sentence[0].upper() + sentence[1:]
                    # print("Corrected sentence:", sentence)
                    sentences[idx] = sentence
                    # print("by:", sentences[idx])

            # print("\n\nFound to translate:", sentences)
            translated = pipe_translate(sentences)
            # print("Translated text:", translated)
            translated_aggregated = translated[0]["translation_text"]
            for sentence in translated[1:]:
                translated_aggregated += " " + sentence["translation_text"]
            i.replace_with(translated_aggregated)

        output_archive.writestr(file_list[x].filename, soup.prettify("utf-8"))
        # pprint(soup.prettify("utf-8"))
    else:
        #For the other file types, simply copy the original content:
        output_archive.writestr(file_list[x].filename, content)

input_archive.close()
output_archive.close()

[<ZipInfo filename='mimetype' filemode='-rw-rw-rw-' file_size=20>,
 <ZipInfo filename='META-INF/container.xml' compress_type=deflate filemode='-rw-rw-rw-' file_size=265 compress_size=180>,
 <ZipInfo filename='OEBPS/content.opf' compress_type=deflate filemode='-rw-rw-rw-' file_size=19038 compress_size=3957>,
 <ZipInfo filename='OEBPS/toc.ncx' compress_type=deflate filemode='-rw-rw-rw-' file_size=11821 compress_size=1846>,
 <ZipInfo filename='OEBPS/Images/00001.jpeg' compress_type=deflate filemode='-rw-rw-rw-' file_size=32367 compress_size=27517>,
 <ZipInfo filename='OEBPS/Images/00002.jpeg' compress_type=deflate filemode='-rw-rw-rw-' file_size=21174 compress_size=20118>,
 <ZipInfo filename='OEBPS/Images/00003.jpeg' compress_type=deflate filemode='-rw-rw-rw-' file_size=57024 compress_size=55598>,
 <ZipInfo filename='OEBPS/Images/00004.jpeg' compress_type=deflate filemode='-rw-rw-rw-' file_size=151403 compress_size=151287>,
 <ZipInfo filename='OEBPS/Images/00005.jpeg' compress_type=deflat

  0%|          | 0/107 [00:00<?, ?it/s]/home/qbateux/pytorch/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 107/107 [44:01<00:00, 24.69s/it]  
